# Proyecto Estructura y Bases de Datos

*   Antonia Morales
*   Ángel Paisano
*   Martín Raffo
*   Esteban Sánchez





# Primera descripción

En el presente proyecto se utiliza la API astroquery para cargar los datos de Gaia usando ADQL (Astronomical Data Query Language) que es el lenguaje que usa Gaia; este es idéntico a SQL, que fue visto en clases, pero con funciones integradas para trabajar datos astronómicos. En principio los datos se almacenan en una tabla astropy, y luego son transformados a dataframes de Pandas para hacerlos accesibles, para posteriormente ser llevados a una base de datos SQL.



# Gaia

In [ ]:
#Instalando astroquery
!pip install astroquery

In [ ]:
#Se importan librerías
from astroquery.gaia import Gaia
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Se aumenta el timeout para que pueda correr tranquilamente el código
Gaia.TIMEOUT = 1e6

In [ ]:
#Se cargan las tablas
tables = Gaia.load_tables(only_names=True)

In [ ]:
#Imprimir nombres de tablas
for table in tables:
    print(table.name)

## ADQL
La muestra de la gran nube de magallanes (siglas en inglés: LMC) se obtiene usando una selección con un radio de 20° con $α,δ$ (81.28°,-67.78°). El trabajo se enfocará en esta región del cielo para procesar datos de la galaxia anteriormente mencionada, estudiando las posiciones en el cielo de las fuentes astronómicas (sources), sus movimientos propios, y las magnitudes en distintos filtros. Este paso está basado en el artículo científico de Luri et al (2021) que se dedica a seleccionar datos de Gaia EDR3 en la región de la LMC.

In [ ]:
# Consulta a Gaia y conversión
job0 = Gaia.launch_job_async("""SELECT TOP 100000 source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag, phot_bp_mean_mag,
                      phot_rp_mean_mag FROM gaiadr2.gaia_source
                      WHERE DISTANCE(POINT(81.28, -69.78),POINT(ra, dec)) < 20
                      AND gaiadr2.gaia_source.parallax IS NOT NULL""")
tabla_gaiadr2 = job0.get_results()  # Resultado en formato Table de Astropy

In [ ]:
tabla_gaiadr2

In [ ]:
# Consulta a Gaia y conversión EDR3
job1 = Gaia.launch_job("""SELECT TOP 100000 source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag, phot_bp_mean_mag,
                      phot_rp_mean_mag FROM gaiaedr3.gaia_source
                      WHERE DISTANCE(POINT(81.28, -69.78),POINT(ra, dec)) < 20
                      AND gaiaedr3.gaia_source.parallax IS NOT NULL""")
tabla_gaiaedr3 = job1.get_results()  # Resultado en formato Table de Astropy

In [ ]:
tabla_gaiaedr3

## Pandas



In [ ]:
df_gaiadr2= tabla_gaiadr2.to_pandas()
df_gaiadr2= df_gaiadr2.rename(columns={"SOURCE_ID": "source_id"})

In [ ]:
df_gaiadr2

In [ ]:
df_gaiaedr2= tabla_gaiaedr3.to_pandas()
df_gaiaedr3= df_gaia3dr3.rename(columns={"SOURCE_ID": "source_id"})

In [ ]:
df_gaiaedr3

# SQL

In [ ]:
# Función para crear las tablas de gaia
def crear_tabla(tabla, nombre_tabla, db_name):
  """
  Esta función construye...

  """
    db = sqlite3.connect(db_name)
    cur = db.cursor()

    ############################
    cur.execute(f"""
        CREATE TABLE {nombre_tabla} (
            source_id INTEGER PRIMARY KEY AUTOINCREMENT,
            ra REAL,
            dec REAL,
            parallax REAL,
            pmra REAL,
            pmdec REAL,
            phot_g_mean_mag REAL,
            phot_bp_mean_mag REAL,
            phot_rp_mean_mag REAL
        );
    """)

    #######################################################
    for i, row in tabla.iterrows():
        cur.execute(f"""
            INSERT OR IGNORE INTO {nombre_tabla} (source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag)
            VALUES (?, ?, ?, ?, ?, ?, ?, ? ,?);
        """, (row['source_id'], row['ra'], row['dec'], row['parallax'], row['pmra'], row['pmdec'], row['phot_g_mean_mag'], row['phot_bp_mean_mag'], row['phot_rp_mean_mag']))

    db.commit()

    # Verificación:
    res = cur.execute(f"PRAGMA table_info({nombre_tabla})")
    if res.fetchall() != []:
        print("Tabla creada correctamente")
        print(res.fetchall())
    else:
        print("Error al crear la tabla")

    db.close()

In [ ]:
#Creación de tablas con la función

crear_tabla(df_gaiadr2, 'gaiadr2', 'gaiadr2.db')

# Querys SQL

In [ ]:
db = sqlite3.connect('gaiadr2.db')
cur = db.cursor()

In [ ]:
res = cur.execute(f"""PRAGMA table_info(gaiadr2)""")
res.fetchall()

In [ ]:
query1= cur.execute(f"""SELECT COUNT(*) FROM gaiadr2""")
query1.fetchall()

# Gráficos

In [ ]:
plt.style.use('dark_background')

In [ ]:
"""
#Variables que utilizaremos para graficar
ra= df_gaiaedr3['ra']
dec= df_gaiaedr3['dec']
pmra= df_gaiaedr3['pmra']
pmdec= df_gaiaedr3['pmdec']
mag_bp= df_test['phot_bp_mean_mag']
mag_rp= df_test['phot_rp_mean_mag']
mag_g= df_test['phot_g_mean_mag']
"""

In [ ]:
fig= plt.figure(figsize=(14,6))
#fig.subplots_adjust(left=0.1, bottom=0.1, top=0.97, right =0.97)

ax1= fig.add_subplot(121)

ax1.scatter(ra, dec, c='cyan', marker='*',s=10, edgecolors= "none") #c=brillo, cmap='plasma',s=(1/brillo)*40)
ax1.scatter(81.28, -69.78, marker="+", facecolor="red",label='Centro aparente', s=40)
ax1.set_xlabel("RA")
ax1.set_ylabel("DEC")
ax1.set_title(" ra vs dec de la Gran Nube de Magallanes")
ax1.grid(ls='--', alpha=0.3)
ax1.legend()

ax2= fig.add_subplot(122)

ax2.scatter(pmra, pmdec, c='cyan', marker='*',s=10, edgecolors= "none") #c=brillo, cmap='plasma',s=(1/brillo)*40)
ax2.set_xlabel("pmra")
ax2.set_ylabel("pmdec")
ax2.set_title("pmra vs pmdec de la Gran Nube de Magallanes")
ax2.grid(ls='--', alpha=0.3)

plt.show()

## Diagrama color magnitud test
BP-RP vs G

In [ ]:
#añadir errorbars y colorbars

fig = plt.figure(1,figsize=(6,6))
ax1= fig.add_subplot(111)
ax1.scatter(mag_bp - mag_rp, mag_g,c='darkred',marker='*',s=10, edgecolors= "none") #c=brillo, cmap='plasma',s=(1/brillo)*40)
#ax1.scatter((mag_bp-mag_rp)[i], mag_g[i],marker="*",edgecolor="none", facecolor="red",s=10)
ax1.set_xlabel("BP-RP")
ax1.set_ylabel("G")
ax1.set_title("Diagrama color magnitud Gran Nubre de Magallanes")
ax1.grid(ls='--', alpha=0.3)
#plt.ylim(16,21.5)
plt.gca().invert_yaxis()

plt.show()